# load model

In [34]:
model_path = 'models/'

resnet_best = 'resnet50_3_tuned1.h5'
incept_best = 'inception1.h5'
nasnet_best = 'NASNetMobile_1.h5'


In [35]:
import tensorflow as tf

resnet_model = tf.keras.models.load_model(model_path + resnet_best)
incept_model = tf.keras.models.load_model(model_path + incept_best)
nasnet_model = tf.keras.models.load_model(model_path + nasnet_best)

In [27]:

def add_prefix(model, prefix: str, custom_objects=None):
    '''Adds a prefix to layers and model name while keeping the pre-trained weights
    Arguments:
        model: a tf.keras model
        prefix: a string that would be added to before each layer name
        custom_objects: if your model consists of custom layers you shoud add them pass them as a dictionary. 
            For more information read the following:
            https://keras.io/guides/serialization_and_saving/#custom-objects
    Returns:
        new_model: a tf.keras model having same weights as the input model.
    '''
    
    config = model.get_config()
    old_to_new = {}
    new_to_old = {}
    
    for layer in config['layers']:
        new_name = prefix + layer['name']
        old_to_new[layer['name']], new_to_old[new_name] = new_name, layer['name']
        layer['name'] = new_name
        layer['config']['name'] = new_name

        if len(layer['inbound_nodes']) > 0:
            for in_node in layer['inbound_nodes'][0]:
                in_node[0] = old_to_new[in_node[0]]
    
    for input_layer in config['input_layers']:
        input_layer[0] = old_to_new[input_layer[0]]
    
    for output_layer in config['output_layers']:
        output_layer[0] = old_to_new[output_layer[0]]
    
    config['name'] = prefix + config['name']
    new_model = tf.keras.Model().from_config(config, custom_objects)
    
    for layer in new_model.layers:
        layer.set_weights(model.get_layer(new_to_old[layer.name]).get_weights())
    
    return new_model


resnet_model2 = add_prefix(resnet_model, 'resnetmj_')

In [28]:
models = [resnet_model2, incept_model, nasnet_model]
model_input = tf.keras.Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = tf.keras.layers.Average()(model_outputs)
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)

# load data

In [5]:
import pickle

folder_path = 'dataset_224_224/'
x_train, y_train, x_val, y_val, x_test, y_test = [], [], [], [], [], []

with open(folder_path+'x_train.pickle', 'rb') as f:
    x_train = pickle.load(f)

with open(folder_path+'y_train.pickle', 'rb') as f:
    y_train = pickle.load(f)

with open(folder_path+'x_val.pickle', 'rb') as f:
    x_val = pickle.load(f)

with open(folder_path+'y_val.pickle', 'rb') as f:
    y_val = pickle.load(f)

with open(folder_path+'x_test.pickle', 'rb') as f:
    x_test = pickle.load(f)

with open(folder_path+'y_test.pickle', 'rb') as f:
    y_test = pickle.load(f)
    
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

(17154, 224, 224, 3) (17154,) (5718, 224, 224, 3) (5718,) (5718, 224, 224, 3) (5718,)


In [6]:
train_images = x_train
train_labels = y_train
val_images = x_val
val_labels = y_val
test_images = x_test
test_labels = y_test

y_train = tf.keras.utils.to_categorical(train_labels, 3)
y_val = tf.keras.utils.to_categorical(val_labels, 3)
y_test = tf.keras.utils.to_categorical(test_labels, 3)
y_train.shape, y_val.shape, y_test.shape

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

(17154, 224, 224, 3) (17154, 3) (5718, 224, 224, 3) (5718, 3) (5718, 224, 224, 3) (5718, 3)


In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

checkpoint_path = "/Users/jeonminjeong/Downloads/알파코/프로젝트/이미지인식/checkpoints/ensamble/cp-4-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=False, # False면 모델 레이어 및 weight값 모두
    save_freq='epoch' # 체크포인트 저장하는 에폭주기 설정 가능
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=0)

model_name = '_Ensamble'
current_time = datetime.datetime.now().strftime("%Y%m%d") # Y : 연도 / m : 월 / d : 일 / H : 시 / M : 분 / S : 초

log_dir = "logs/" + current_time + model_name
board_ensamble = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1) # epoch마다 히스토그램 계산

In [30]:
ensemble_model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
hist_classifi_6 = ensemble_model.fit(train_images, y_train, epochs = 30, validation_data=(val_images, y_val), batch_size=125
                            , callbacks=[cp_callback, early, board_ensamble, reduce_lr])

In [ ]:
print(hist_classifi_6.history['loss'])
print(hist_classifi_6.history['accuracy'])
print(hist_classifi_6.history['val_loss'])
print(hist_classifi_6.history['val_accuracy'])

import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist_classifi_6.history['loss'], 'y', label='train loss')
loss_ax.plot(hist_classifi_6.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(hist_classifi_6.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist_classifi_6.history['val_accuracy'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='lower left')

plt.show()


In [31]:
#모델 검증
test_loss, test_acc = ensemble_model.evaluate(test_images, y_test, verbose=2)
print('loss: ', test_loss, ', acc: ', test_acc)  

2022-12-01 17:15:08.101265: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


179/179 - 201s - loss: 0.5425 - accuracy: 0.8596 - 201s/epoch - 1s/step
loss:  0.542522132396698 , acc:  0.859566330909729


In [ ]:
ensemble_model.save('models/ensemble.h5')

In [10]:
%load_ext tensorboard
%tensorboard --logdir logs

In [36]:
resnet_model.save('models/resnet50', save_format="tf")

INFO:tensorflow:Assets written to: models/resnet50/assets


INFO:tensorflow:Assets written to: models/resnet50/assets
